In [1]:
using Laplacians

In [2]:
include("../../../src/isotonicIPMSolverDebug.jl")
include("../../../src/samplingTreeSolver.jl")

constructLowerTriangularMat (generic function with 1 method)

In [3]:
sqroot = 300
n = sqroot^2
A = grid2(sqroot,sqroot)
A = triu(A);
v = randn(n)+collect(1:n)/2;
isoTestValues = collect(1:n);
newOrder = randperm(n)
permMat = (speye(n))[1:n,newOrder];
A = permMat*A*permMat';
v = permMat*v;

In [4]:
@time (x,acc,itercount) = isotonicIPMrelEps(A,v,0.01);

*****************

rho = 10.41060304582247
Average number of multiedges = 5.2265338126579515
Maximum number of multiedges = 10.410603085816941
Time to build the tree and compute the stretch: elapsed time: 0.73053874 seconds

The total size of the linked list data structure should be at most 4612225
The actual size is 107820

My build time: elapsed time: 2.332520782 seconds
PCG stopped at maxtime.
PCG stopped after: 2144 iterations with relative error 0.00019931454593709812.
My solve time: elapsed time: 20.150098119 seconds
0.00019931454557176696

Dan's build time: elapsed time: 0.728350792 seconds
PCG stopped after: 1492 iterations with relative error 8.991139813814344e-7.
Dan's solve time: elapsed time: 12.491721303 seconds
9.152913486553347e-7

*****************

rho = 10.41060304582247
Average number of multiedges = 5.22656293258755
Maximum number of multiedges = 10.410603096457065
Time to build the tree and compute the stretch: elapsed time: 0.812931283 seconds

The total size of t

In [5]:
a = copy(x); b = copy(acc);
itercount

"hard graph"

In [3]:
a = readFromFile("3.txt");

In [7]:
n = a.n

90000

In [8]:
b = rand(n); b = b - mean(b);

In [9]:
@time f = samplingSolver(a, tol=1e-6,maxits=120,verbose=true,
eps = 0.5, sampConst = 0.02, beta = 1000.0)
@time norm(lap(a) * f(b) - b) / norm(b)

rho = 10.41060304582247
Average number of multiedges = 5.2266946715106295
Maximum number of multiedges = 10.410603085816941
Time to build the tree and compute the stretch: elapsed time: 0.804177652 seconds

The total size of the linked list data structure should be at most 4612282
The actual size is 107820

  1.878346 seconds (13.87 M allocations: 794.532 MB, 14.49% gc time)
PCG stopped after: 120 iterations with relative error 8.752336662524488e-6.
  1.649896 seconds (22.19 M allocations: 870.554 MB, 8.22% gc time)


8.75232874891605e-6

In [10]:
@time danf = augTreeSolver(lap(a) + speye(a.n) * 1e-15, tol=1e-6,maxits=1000,verbose=true,treeAlg = akpw)
@time nrm = norm((lap(a) + speye(a.n) * 1e-15) * danf(b) - b) / norm(b)

  0.882606 seconds (1.34 M allocations: 297.175 MB, 17.52% gc time)
PCG stopped after: 1000 iterations with relative error 1.3922058369059718.
  9.862259 seconds (113.16 k allocations: 5.406 GB, 8.40% gc time)


1.3922058368847192

In [11]:
norm(lap(a) * (danf(b) - f(b))) / norm(b)

PCG stopped after: 1000 iterations with relative error 1.3922058369059718.
PCG stopped after: 120 iterations with relative error 8.752336662524488e-6.


1.3922059041510164

In [16]:
r = lap(a) * f(b) - b
t = r' * foneStep(r)
t / (b'*foneStep(b))

PCG stopped after: 120 iterations with relative error 8.752336662524488e-6.


1x1 Array{Float64,2}:
 6.8801e-11

In [17]:
r = lap(a) * danf(b) - b
t = r' * foneStep(r)
t / (b' * foneStep(b))

PCG stopped after: 1000 iterations with relative error 1.3922058369059718.


1x1 Array{Float64,2}:
 0.681964

In [18]:
using PyAMG

In [25]:
@time x = PyAMG.solve(lap(a) + speye(a.n) * 1e-15, b, tol = 1e-6, maxiter = 100);

  1.882712 seconds (323 allocations: 38.409 MB, 0.50% gc time)
  Detected a Hermitian matrix
    maxiter = 100
    iteration 1
    iteration 2
    iteration 3
    iteration 4
    iteration 5
    iteration 6
    iteration 7
    iteration 8
    iteration 9
    iteration 10
    iteration 11
  Residual reduction ||r_k||/||r_0|| = 1.10e-05


In [26]:
r = lap(a) * x - b
t = r' * foneStep(r)
t / (b' * foneStep(b))

1x1 Array{Float64,2}:
 1.29136e-10

In [49]:
maximum(abs(b))

0.022106446804653984

In [8]:
writeToFile("2.txt", a)

In [13]:
@time foneStep,gOp,U,d,_,cn,cntime,stretches = buildSolver(a, eps = 0.5, sampConst = 0.02, beta = 1000.0, returnCN=true);

rho = 10.41060304582247
Average number of multiedges = 5.2266946715106295
Maximum number of multiedges = 10.410603085816941
Time to build the tree and compute the stretch: elapsed time: 0.859398843 seconds

The total size of the linked list data structure should be at most 4612282
The actual size is 107820

computing the condition number takes: elapsed time: 6.321184471 seconds
elapsed time: 6.355505206 seconds
  7.815653 seconds (65.49 M allocations: 2.083 GB, 6.50% gc time)


In [14]:
checkError(gOp)

-0.951000301968924

In [15]:
cn

(317.88235820595753,339.4643006921061)

1x90000 Array{Float64,2}:
 -0.0147591  -0.0147761  -0.0147898  …  3.69294e-7  -5.36868e-7  -3.9492e-5

In [28]:
@time eigs(a + speye(n) * 0.001, nev = 3, which = :LM)[1]

LoadError: LoadError: InterruptException:
while loading In[28], in expression starting on line 155

In [39]:
@time eigs(a + speye(n) * 0.001, nev = 3, which = :SM)[1]

601.159546 seconds (72.84 k allocations: 10.255 GB, 0.29% gc time)


0.0